In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd

In [2]:
cropped_images_features = pd.read_parquet('./cropped_images_features.parquet')
not_cropped_images_features = pd.read_parquet('./not_cropped_images_features.parquet')
# combine the two dataframes
df = pd.concat([cropped_images_features, not_cropped_images_features])
# mix the rows
df = df.sample(frac=1).reset_index(drop=True)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['label']), df['label'], test_size=0.2, random_state=42)

In [4]:
model = Sequential([
    Flatten(input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
model_checkpoint = 'tmp/model.keras'
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), callbacks=[ModelCheckpoint(model_checkpoint, save_best_only=True)])

Epoch 1/20
332/332 [==============================] - 13s 17ms/step - loss: 0.4065 - accuracy: 0.8144 - val_loss: 0.2524 - val_accuracy: 0.8750
Epoch 2/20
332/332 [==============================] - 5s 15ms/step - loss: 0.2332 - accuracy: 0.8978 - val_loss: 0.1560 - val_accuracy: 0.9347
Epoch 3/20
332/332 [==============================] - 5s 15ms/step - loss: 0.1982 - accuracy: 0.9153 - val_loss: 0.1704 - val_accuracy: 0.9268
Epoch 4/20
332/332 [==============================] - 5s 15ms/step - loss: 0.1839 - accuracy: 0.9210 - val_loss: 0.1341 - val_accuracy: 0.9434
Epoch 5/20
332/332 [==============================] - 5s 16ms/step - loss: 0.1703 - accuracy: 0.9258 - val_loss: 0.1329 - val_accuracy: 0.9402
Epoch 6/20
332/332 [==============================] - 5s 14ms/step - loss: 0.1667 - accuracy: 0.9293 - val_loss: 0.1372 - val_accuracy: 0.9357
Epoch 7/20
332/332 [==============================] - 5s 16ms/step - loss: 0.1633 - accuracy: 0.9307 - val_loss: 0.1269 - val_accuracy: 0.945

In [6]:
model.load_weights(model_checkpoint)
model.evaluate(X_test, y_test)

166/166 [==============================] - 1s 7ms/step - loss: 0.1082 - accuracy: 0.9604


[0.10824327170848846, 0.9603922963142395]